In [2]:
!java -version

java version "19.0.1" 2022-10-18
Java(TM) SE Runtime Environment (build 19.0.1+10-21)
Java HotSpot(TM) 64-Bit Server VM (build 19.0.1+10-21, mixed mode, sharing)


# Get PDF Data

In [3]:
import pandas as pd
import numpy as np
import tabula
from tabulate import tabulate
import re


### Declare Column Names and Read in Data

In [4]:
bach_cols_2021 = [
    'cip_code',
    'total_graduates',
    'knowledge_rate',
    'career_outcomes_perc',
    'total_perc_employed_overall',
    'total_perc_employed_full_time',
    'total_perc_employed_part_time',
    'perc_standard_employment_overall',
    'perc_standard_full_time',
    'perc_standard_part_time',
    'perc_entrepreneur_overall',
    'perc_entrepreneur_full_time',
    'perc_entrepreneur_part_time',
    'perc_temp_contract_overall',
    'perc_temp_contract_full_time',
    'perc_temp_contract_part_time',
    'perc_freelance_overall',
    'perc_freelance_full_time',
    'perc_freelance_part_time',
    'perc_fellowship_intern_overall',
    'perc_fellowship_intern_full_time',
    'perc_fellowship_intern_part_time',
    'perc_service',
    'perc_military',
    'perc_continuing_ed',
    'perc_still_seeking_overall',
    'perc_seeking_employment',
    'perc_seeking_continuing_ed',
    'perc_not_seeking',
    'mean_starting_salary'
    ]

mast_cols_2021 = [
    'cip_code',
    'total_graduates',
    'knowledge_rate',
    'career_outcomes_perc',
    'total_perc_employed_overall',
    'total_perc_employed_full_time',
    'total_perc_employed_part_time',
    'perc_standard_employment_overall',
    'perc_standard_full_time',
    'perc_standard_part_time',
    'perc_faculty_overall',
    'perc_faculty_tenure',
    'perc_faculty_non_tenure',
    'perc_entrepreneur_overall',
    'perc_entrepreneur_full_time',
    'perc_entrepreneur_part_time',
    'perc_temp_contract_overall',
    'perc_temp_contract_full_time',
    'perc_temp_contract_part_time',
    'perc_freelance_overall',
    'perc_freelance_full_time',
    'perc_freelance_part_time',
    'perc_fellowship_intern_overall',
    'perc_fellowship_intern_full_time',
    'perc_fellowship_intern_part_time',
    'perc_service',
    'perc_military',
    'perc_continuing_ed',
    'perc_still_seeking_overall',
    'perc_seeking_employment',
    'perc_seeking_continuing_ed',
    'perc_not_seeking',
    'mean_starting_salary'
    ]

In [5]:
bach_cols_2018 = list(np.append(bach_cols_2021[:-5], bach_cols_2021[-4:]))
bach_cols_2017 = bach_cols_2018
bach_cols_2016 = bach_cols_2018

mast_cols_2016 = list(np.append(mast_cols_2021[:10], mast_cols_2021[13:]))
mast_cols_2016 = list(np.append(mast_cols_2016[:-5], mast_cols_2016[-4:]))

In [6]:
bach_2021 = tabula.read_pdf('pdf_data\\2021.pdf', pages='16-73')
mast_2021 = tabula.read_pdf('pdf_data\\2021.pdf', pages='78-125')
bach_2020 = tabula.read_pdf('pdf_data\\2020.pdf', pages='17-67', encoding='cp1252')
mast_2020 = tabula.read_pdf('pdf_data\\2020.pdf', pages='72-117', encoding='cp1252')
bach_2018 = tabula.read_pdf('pdf_data\\2018.pdf', pages='24-71')
mast_2018 = tabula.read_pdf('pdf_data\\2018.pdf', pages='73-116')
bach_2017 = tabula.read_pdf('pdf_data\\2017.pdf', pages='25-70')
mast_2017 = tabula.read_pdf('pdf_data\\2017.pdf', pages='72-114')
bach_2016 = tabula.read_pdf('pdf_data\\2016.pdf', pages='25-70')
mast_2016 = tabula.read_pdf('pdf_data\\2016.pdf', pages='72-115')

In [7]:
bach_dict = {'2016':(bach_2016, bach_cols_2016), '2017':(bach_2017, bach_cols_2017), '2018':(bach_2018, bach_cols_2018), '2020':(bach_2020, bach_cols_2018), '2021':(bach_2021, bach_cols_2021)}
mast_dict = {'2016':(mast_2016, mast_cols_2016), '2017':(mast_2017, mast_cols_2021), '2018':(mast_2018, mast_cols_2021), '2020':(mast_2020, mast_cols_2021), '2021':(mast_2021, mast_cols_2021)}

### Create Functions for Cleaning and Combining

In [24]:
def clean_and_combine(df_list, year, cols):
    #create empty df to combine into
    total_df = pd.DataFrame()

    #initial cleaning, get rid of unwanted empty columns
    new_df_list = []
    for i in df_list:
        df = i
        for x in range(1,10):
            if f'Unnamed: {x}' in df.columns:
                df = df.drop(columns=f'Unnamed: {x}')
        if (len(df) > 1) and (len(df.columns) > 3):
            new_df_list.append(df[(df[df.columns[-2]].notna())&(df[df.columns[-3]].notna())])
        elif (len(df) > 1) and (len(df.columns) == 3):
            new_df_list.append(df[df[df.columns[-2]].notna()])
        elif (len(df) > 1) and (len(df.columns) == 2):
            new_df_list.append(df[df[df.columns[-1]].notna()])

    #shift misplaced rows
    middle_df_list = []
    for i in new_df_list:
        df = i.reset_index(drop=True)
        for ind,r in df.iterrows():
            if type(df.at[ind, df.columns[-1]]) == float:
                row_vals = ['NA']
                row_vals.extend(list(df.iloc[ind][:-1]))
                df.iloc[ind] = row_vals
        if len(df) > 3:
            middle_df_list.append(df)

    #split unnecessarily combined columns
    final_df_list = []
    for i in middle_df_list:
        df = i
        if type(df.at[0, df.columns[-1]]) == float:
            row_vals = ['NA']
            row_vals.extend(list(df.iloc[0][:-1]))
            df.iloc[0] = row_vals
        try:
            for x in df.columns:
                if ('Unnamed' not in x) and (' ' in x):
                    col_to_sep = x
                    col_sep = x.split(' ')
                    df[col_sep] = df[col_to_sep].str.split(' ',n=1,expand=True)
                    df = df.drop(columns=col_to_sep)
        except:
            continue
        final_df_list.append(df)

    #create copy of each df
    #shift columns if needed, transpose, and combine total_df
    for i in final_df_list:
        try:
            if len(i) >0:
                copy = i.copy()
                if copy.columns[0] != 'Unnamed: 0':
                    columns = copy.columns[:-1].insert(0, 'Unnamed: 0')
                    copy = copy.set_axis(columns,axis=1)
                copy = copy.iloc[-(len(cols)-1):].drop(columns='Unnamed: 0').T.reset_index()
                copy = copy.set_axis(cols, axis=1)
                total_df = pd.concat([total_df, copy], ignore_index=True)
        except:
            continue
    total_df['year'] = year

    #drop rows with null values
    total_df = total_df.dropna()

    #clean data and change dtypes to numeric
    total_df = total_df.loc[total_df['mean_starting_salary'] !='NA']
    total_grads = []
    for i in total_df['total_graduates'].values:
        try:
            total_grads.append(int(re.sub('[^1-9]', '', i)))
        except:
            total_grads.append(0)
    total_df['total_graduates'] = total_grads
    mean_sals = []
    for i in total_df['mean_starting_salary'].values:
        try:
            mean_sals.append(int(re.sub('[^1-9]', '', i)))
        except:
            mean_sals.append(re.sub('[^1-9]', '', i))
    total_df['mean_starting_salary'] = mean_sals
    cips = []
    for i in total_df['cip_code'].values:
        cips.append(re.sub('\.', '', i))
    total_df['cip_code'] = cips
    for i in total_df.columns[2:-2]:
        col_vals = []
        for x in total_df[i].values:
            try:
                col_vals.append(float(re.sub('[^1-9.]','',x)))
            except:
                col_vals.append(float(0))
        total_df[i] = col_vals

    #return cleaned and combined df
    return total_df

In [48]:
def get_all_data(df_dict, grad_threshold, kr_threshold):
    #create empty dataframe to combine into
    combined = pd.DataFrame()

    #add all desired years
    for i in df_dict.keys():
        year_df = clean_and_combine(df_dict[i][0], int(i), df_dict[i][1])
        combined = pd.concat([combined, year_df], ignore_index=True)

    #only keep data that meets desired graduate and knowledge rate criteria
    combined = combined.loc[(combined['total_graduates'] >= grad_threshold) & (combined['knowledge_rate'] >= kr_threshold) & (combined['cip_code'].str.len()>2)].reset_index(drop=True)
    return combined

### Call Functions with Desired Thresholds

In [49]:
all_bach = get_all_data(bach_dict, 500, 60)
all_mast = get_all_data(mast_dict, 300, 60)

# Generate Lists for Curricula Scraping

### Read in Field-of-Study Data

In [52]:
bach_codes = all_bach.cip_code.unique().astype(int)
mast_codes = all_mast.cip_code.unique().astype(int)
df = pd.read_csv('Most-Recent-Cohorts-Field-of-Study.csv')

### Define List Generation Function

In [57]:
def gen_list(codes, level):
    df_lim = df.loc[(df['CIPCODE'].isin(codes))&(df['CREDLEV']==level)&(df['CONTROL'].isin(['Public', 'Private, nonprofit']))&(df['UNITID'].notna())]

    list_df = pd.DataFrame()

    for i in codes:
        sort_df = df_lim.loc[df_lim['CIPCODE']==i, ['CIPCODE', 'CIPDESC', 'INSTNM']].sample(10)
        list_df = pd.concat([list_df, sort_df], ignore_index=True)
    
    return list_df


### Call Function and Read Out to CSV

In [58]:
bach = gen_list(bach_codes, 3)
mast = gen_list(mast_codes, 5)

In [59]:
bach.to_csv('bach_programs.csv')
mast.to_csv('mast_programs.csv')